### Step 1: Initialize Parameters
This step sets up the widgets to receive parameters.

In [0]:
# Set up widgets to receive parameters
dbutils.widgets.text("business_unit", "")
dbutils.widgets.text("source_path", "")
dbutils.widgets.text("destination_path", "")
dbutils.widgets.text("source_name", "")
dbutils.widgets.text("transformation_rules", "{}")

# Retrieve the parameter values
business_unit = dbutils.widgets.get("business_unit")
source_path = dbutils.widgets.get("source_path")
destination_path = dbutils.widgets.get("destination_path")
source_name = dbutils.widgets.get("source_name")
transformation_rules = dbutils.widgets.get("transformation_rules")

# Convert transformation rules to JSON (if provided)
import json
try:
    rules = json.loads(transformation_rules)
except Exception as e:
    rules = {}


### Step 2: Read Data from Bronze Layer
This step reads the source data from the Bronze layer.


In [0]:
# Build the file path for the Bronze layer file (assuming Parquet format)
spark.conf.set(
    "fs.azure.account.key.stgdatapoc.dfs.core.windows.net",
    "QD5wJQ6uA9+sdAdrFma1Qvn/GhfQN5Ivs7rIqUxUy4tfS//N+TPaeU1xpEWMHJaKtvzndAa2EHGj+ASt32QmjQ=="
)
bronze_file = f"abfss://{source_path}@stgdatapoc.dfs.core.windows.net/{business_unit}/{source_name}.parquet"

# Read the data from the Bronze layer
df = spark.read.parquet(bronze_file)


### Step 3: Apply Common Transformations
Here we apply general transformations such as removing duplicates and filling null values.


In [0]:
# Apply common transformations: remove duplicates
df_transformed = df.dropDuplicates()

# Optionally fill null values if defined in transformation rules
if rules.get("fill_nulls"):
    df_transformed = df_transformed.na.fill(rules["fill_nulls"])

# Optionally rename columns if specified in the transformation rules
if rules.get("rename_columns"):
    for old_name, new_name in rules["rename_columns"].items():
        df_transformed = df_transformed.withColumnRenamed(old_name, new_name)


### Step 4: Write Transformed Data to Silver Layer
Finally, write the transformed data to the Silver layer.


In [0]:
# Build the file path for the Silver layer output
silver_file = f"abfss://{destination_path}@stgdatapoc.dfs.core.windows.net/{business_unit}/{source_name}.parquet"

# Write the transformed data to the Silver layer (overwrite mode)
df_transformed.write.mode("overwrite").parquet(silver_file)

# Exit the notebook indicating success
dbutils.notebook.exit("Success")
